# Disparities in Mental Health Service Usage during the COVID-19 Pandemic

We examined the gender and racial/ethnic disapirites in mental health service usagecontrolling for internalizing problem severity with logistic regression and SMOTE sampling for unbalanced classes. 

We compared the predictive effects of demographic varialbes combined with four separate internalizing domains vs. one aggregated overall internalizing severity on mental health service usage. 

In [1]:
#Load pacakges
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTENC  
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline
from numpy import mean
from numpy import std
from numpy import round
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp

In [2]:
#Customized preprocessing
class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_features, categorical_features):
        self.numeric_features = numeric_features
        self.categorical_features = categorical_features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X[self.numeric_features]
        #Dummy code categorical varialbes 
        X_copy = X_copy.join(pd.get_dummies(X[self.categorical_features], drop_first=True)) 
        return X_copy 

In [3]:
#Customized function to create interaction terms
class InteractionTerm(BaseEstimator, TransformerMixin):
    def __init__(self, set1, set2):
        self.set1 = set1
        self.set2 = set2
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_copy = X.copy()
        for var1 in self.set1:
            for var2 in self.set2:
                X_copy[var1+':'+var2] = X_copy[var1]*X_copy[var2]
        return X_copy 

In [4]:
#Read Data
df = pd.read_csv('../Results/txData_naOmit2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  909 non-null    object 
 1   age                 909 non-null    int64  
 2   gender              909 non-null    int64  
 3   race                909 non-null    int64  
 4   eth                 909 non-null    int64  
 5   baseline            909 non-null    float64
 6   PHQ_mean            909 non-null    float64
 7   GAD_mean            909 non-null    float64
 8   IUS_mean            909 non-null    float64
 9   PTSD_mean           909 non-null    float64
 10  overallSev          909 non-null    float64
 11  txHist_now          909 non-null    int64  
 12  txHist_past         909 non-null    int64  
 13  txHist_wanted       909 non-null    int64  
 14  txHist_neverWanted  909 non-null    int64  
dtypes: float64(6), int64(8), object(1)
memory usage: 106.6+ K

In [14]:
#Specify Data Type
categoricalVar = ['gender', 'race', 'eth', 'txHist_now', 'txHist_past', 'txHist_wanted', 'txHist_neverWanted']
df[categoricalVar] = df[categoricalVar].astype('category')
df.info()
df.head()
#Gender 1 = Men; 2 = Women; 3 = Other specified gender
#Race 1 = White, 2 = Black; 4 = Asian; 6 = Other
#Eth 0 = Non-Hispanic/Latinx; 1 = Hisapnic/Latinx

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   id                  909 non-null    object  
 1   age                 909 non-null    int64   
 2   gender              909 non-null    category
 3   race                909 non-null    category
 4   eth                 909 non-null    category
 5   baseline            909 non-null    float64 
 6   PHQ_mean            909 non-null    float64 
 7   GAD_mean            909 non-null    float64 
 8   IUS_mean            909 non-null    float64 
 9   PTSD_mean           909 non-null    float64 
 10  overallSev          909 non-null    float64 
 11  txHist_now          909 non-null    category
 12  txHist_past         909 non-null    category
 13  txHist_wanted       909 non-null    category
 14  txHist_neverWanted  909 non-null    category
dtypes: category(7), float64(6), int64(1), ob

,id,age,gender,race,eth,baseline,PHQ_mean,GAD_mean,IUS_mean,PTSD_mean,overallSev,txHist_now,txHist_past,txHist_wanted,txHist_neverWanted
0,R_0HvLbcWuG0qvY'7,19,2,2,0,0.281137,-0.436839,-0.392727,0.699956,-1.268364,-0.454732,0,1,0,0
1,R_3CKvhPmYR'BE6wo,26,2,1,0,-0.661865,1.702294,1.018119,1.377332,0.105005,1.347011,1,1,0,0
2,R_6Du25nbgEoaAQBH,19,2,1,1,-0.661865,0.561423,-0.235966,0.119348,1.478373,0.627447,0,0,1,0
3,R_PLs6HbYPMnE4vgl,21,1,6,0,1.224139,-0.436839,-0.079206,1.474100,1.478373,0.795428,0,0,1,0
4,R_1roFlWXv3P94yt6,19,1,1,0,-0.661865,-0.579447,-0.549488,-0.364492,-1.268364,-0.893623,0,0,1,0


In [6]:
#Standardized Continuous Varaibles (exclude age)
stdsc = StandardScaler()
df[df.select_dtypes(['float']).columns] = stdsc.fit_transform(df.select_dtypes(['float']))

In [7]:
#Define subsets
X1 = df[['gender', 'race', 'eth', 'PHQ_mean', 'GAD_mean', 'IUS_mean', 'PTSD_mean']] #Demographic varialbes with four separate interalizing domains
X2 = df[['gender', 'race', 'eth', 'overallSev']] #Demographic varialbes with one aggregated overall internalzing severity
y = df['txHist_now'].astype('category') #Outcome variable: treatment usage 

#Split data into training and testing sets
n_splits = 10

for rnd in range(100):
    skf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state=rnd)
    i = 0
    for train_index, test_index in skf.split(X1, y):
        #/print("TRAIN:", train_index, "TEST:", test_index)
        locals()[f"X1_train{rnd}_{i}"] = X1.iloc[train_index]
        locals()[f"X1_test{rnd}_{i}"] = X1.iloc[test_index]
        locals()[f"X2_train{rnd}_{i}"] = X2.iloc[train_index]
        locals()[f"X2_test{rnd}_{i}"] = X2.iloc[test_index]
        locals()[f"y_train{rnd}_{i}"] = y.iloc[train_index]
        locals()[f"y_test{rnd}_{i}"] = y.iloc[test_index]
        i += 1


In [8]:
#Smote sampling for unbalanced class (i.e., most people did not use mental health services)
smote = SMOTENC(categorical_features=[0,1,2], random_state=1)

#Define preprocessing 
preprocess1 = Preprocessor(['PHQ_mean','GAD_mean','IUS_mean','PTSD_mean'],
                          ['gender','race','eth'])
preprocess2 = Preprocessor(['overallSev'],
                          ['gender','race','eth'])


#Define interaction terms
interaction1 = InteractionTerm(['PHQ_mean','GAD_mean','IUS_mean','PTSD_mean'], 
                                    ['gender_2','gender_3','race_2','race_4','race_6','eth_1'])
interaction2 = InteractionTerm(['overallSev'], 
                                    ['gender_2','gender_3','race_2','race_4','race_6','eth_1'])

#Define model
model = LogisticRegression(penalty = 'none', max_iter=1000, random_state=1)

performanceM1 = [] #Predictors: Demographic Variables
performanceM2 = [] #Predictors: 4 Separate Interalizing Domains
performanceM3 = [] #Predictors: Aggregate overall internalizing severity 
performanceM4 = [] #Predictors: Demographic Var. + 4 Separate Interalizing Domains
performanceM5 = [] #Predictors: Demographic Var. + Aggregate overall internalizing severity 
performanceM6 = [] #Predictors: Demographic Var. * 4 Separate Interalizing Domains
performanceM7 = [] #Predictors: Demographic Var. * Aggregate overall internalizing severity 

for rnd in range(100):
    for i in range(n_splits):

        X1_train = locals()[f"X1_train{rnd}_{i}"]
        X2_train = locals()[f"X2_train{rnd}_{i}"]
        y_train = locals()[f"y_train{rnd}_{i}"]
        X1_test = locals()[f"X1_test{rnd}_{i}"]
        X2_test = locals()[f"X2_test{rnd}_{i}"]
        y_test = locals()[f"y_test{rnd}_{i}"]

        X1_test_pre = preprocess1.fit_transform(X1_test)
        X2_test_pre = preprocess2.fit_transform(X2_test)

        X1_test_pre_int = interaction1.fit_transform(X1_test_pre)
        X2_test_pre_int = interaction2.fit_transform(X2_test_pre)

        X1_train_sm, y_train_sm = smote.fit_resample(X1_train, y_train)
        X2_train_sm, y_train_sm = smote.fit_resample(X2_train, y_train)

        X1_train_sm_pre = preprocess1.fit_transform(X1_train_sm)
        X2_train_sm_pre = preprocess2.fit_transform(X2_train_sm)

        X1_train_sm_pre_int = interaction1.fit_transform(X1_train_sm_pre)
        X2_train_sm_pre_int = interaction2.fit_transform(X2_train_sm_pre)

        predM1 = model.fit(X1_train_sm_pre[['gender_2','gender_3','race_2','race_4','race_6','eth_1']], 
                          y_train_sm).predict(X1_test_pre[['gender_2','gender_3','race_2','race_4','race_6','eth_1']])
        predM2 = model.fit(X1_train_sm_pre[['PHQ_mean','GAD_mean','IUS_mean','PTSD_mean']], 
                          y_train_sm).predict(X1_test_pre[['PHQ_mean','GAD_mean','IUS_mean','PTSD_mean']])
        predM3 = model.fit(X2_train_sm_pre[['overallSev']], y_train_sm).predict(X2_test_pre[['overallSev']])

        predM4 = model.fit(X1_train_sm_pre, y_train_sm).predict(X1_test_pre)
        predM5 = model.fit(X2_train_sm_pre, y_train_sm).predict(X2_test_pre)
        predM6 = model.fit(X1_train_sm_pre_int, y_train_sm).predict(X1_test_pre_int)
        predM7 = model.fit(X2_train_sm_pre_int, y_train_sm).predict(X2_test_pre_int)

        performanceM1.append(balanced_accuracy_score(y_test, predM1))
        performanceM2.append(balanced_accuracy_score(y_test, predM2))
        performanceM3.append(balanced_accuracy_score(y_test, predM3))
        performanceM4.append(balanced_accuracy_score(y_test, predM4))
        performanceM5.append(balanced_accuracy_score(y_test, predM5))
        performanceM6.append(balanced_accuracy_score(y_test, predM6))
        performanceM7.append(balanced_accuracy_score(y_test, predM7))

print(
      "M1(DEM) %.4f" % mean(performanceM1), "(%.4f)\n" % std(performanceM1),
      "M2(4INT) %.4f" % mean(performanceM2), "(%.4f)\n" % std(performanceM2),
      "M3(1INT) %.4f" % mean(performanceM3), "(%.4f)\n" % std(performanceM3),
      "M4(DEM+4INT) %.4f" % mean(performanceM4), "(%.4f)\n" % std(performanceM4),
      "M5(DEM+1INT)%.4f" % mean(performanceM5), "(%.4f)\n" % std(performanceM5),
      "M6(DEM*4INT) %.4f" % mean(performanceM6), "(%.4f)\n" % std(performanceM6),
      "M7(DEM*1INT) %.4f" % mean(performanceM7), "(%.4f)\n" % std(performanceM7))



M1(DEM) 0.5993 (0.0565)
 M2(4INT) 0.5247 (0.0578)
 M3(1INT) 0.5517 (0.0579)
 M4(DEM+4INT) 0.5987 (0.0557)
 M5(DEM+1INT)0.6001 (0.0542)
 M6(DEM*4INT) 0.5877 (0.0560)
 M7(DEM*1INT) 0.5842 (0.0551)



In [9]:
#T-tests comparing M1 to chance level
ttest_1samp(performanceM1, 0.5)

Ttest_1sampResult(statistic=55.49889925139725, pvalue=1.851379041187304e-307)

The result indicates that M1 performed better than chance. 

In [10]:
print('\nT-tests comparing M1(DEM) and M5(DEM+1INT)')
print(ttest_ind(performanceM1, performanceM5)) 
##The additional main effect of aggreagtted internlizing severity did not significantly improve predication accuracy 



T-tests comparing M1(DEM) and M5(DEM+1INT)
Ttest_indResult(statistic=-0.3315007612411852, pvalue=0.740301032304172)


Overall, the results indicate demographcs factors are more robust predictors of mental health treatment use than internalizing psychiatirc problems, such as depression, anxiety, PTSD, and intolerance of uncertainty.

## Final Model

According to the results above, the final model is the model using demographic variables to predict mental health services use without considering psychiatric symptom severity.

In [11]:
final_model = model.fit(X1_train_sm_pre[['gender_2','gender_3','race_2','race_4','race_6','eth_1']], 
                          y_train_sm)
final_model.coef_

array([[ 1.01011608, -0.5285973 , -1.42999059, -2.38112951, -1.86203444,
        -1.40157919]])

According to the coeeficients, women (gender_2) showed higher odds in using mental health services than men.  
Other-specified gender (gender_3) showed lower odds in using mental health services than men.

People of color (race_2: Black, race_4: Asian; race_6: Others, eth_1: Hispanic/Latinx) showed lower odds in using mental heatlh services than White. Furhter, the Asian American population has the lowest mental heatlh services use rate compared to other racial/ethnic groups. 

In [13]:
#Save SMOTE smple data for outer use 
smote = SMOTENC(categorical_features=[0,1,2], random_state=1)
X2_sm, y_sm = smote.fit_resample(X2, y)
data4Glm = X2_sm
data4Glm['txHist_now'] = y_sm
data4Glm.to_csv('../Data/data4Glm.csv', index = False)